<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/FT_GEMINI_NASA_VERTEXAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env -q
!pip install google-generativeai -q
!pip install rouge-score -q


In [ ]:
from vertexai.preview.tuning import sft
import vertexai
import os
from google.colab import auth
import colab_env
import time

# Project details (replace with your values if not using env vars)
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
REGION = os.environ.get("GOOGLE_CLOUD_REGION")
BUCKET_NAME = os.environ.get("GOOGLE_CLOUD_BUCKET_NAME")
STAGING_BUCKET = f"gs://{BUCKET_NAME}/staging"

# Authentication and Initialization
auth.authenticate_user()
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Define your tuning parameters
BASE_MODEL = "gemini-2.0-flash-001"  # Using Gemini 2.0 Flash

TRAIN_DATASET_URI = f"gs://{BUCKET_NAME}/cmapss_FD004_train_text.jsonl"  # Path to your training data in JSONL format
VALIDATION_DATASET_URI = f"gs://{BUCKET_NAME}/cmapss_FD004_test_text.jsonl"  # Path to your validation data in JSONL format
TUNED_MODEL_DISPLAY_NAME = "cmapss-text-tuned-gemini-2.0-flash-001"
EPOCHS = 10  # Adjust as needed
LEARNING_RATE_MULTIPLIER = 1.0  # Adjust as needed



# Start the fine-tuning job
try:
    sft_tuning_job = sft.train(
        source_model=BASE_MODEL,
        train_dataset=TRAIN_DATASET_URI,
        validation_dataset=VALIDATION_DATASET_URI,
        tuned_model_display_name=TUNED_MODEL_DISPLAY_NAME,
        epochs=EPOCHS,
        learning_rate_multiplier=LEARNING_RATE_MULTIPLIER,
    )


    print(f"Tuning job started: {sft_tuning_job.resource_name}")

    # Periodically check the job status until it's complete
    while True:
        job_status = sft_tuning_job.state  # Get the job's state directly

        if job_status in ("SUCCEEDED", "FAILED", "CANCELLED"):
            break  # Exit the loop if the job is finished

        print(f"Job status: {job_status}, waiting...")
        time.sleep(60)  # Wait for 60 seconds before checking again

    print(f"Tuning job completed with status: {job_status}. Resource name: {sft_tuning_job.resource_name}")



except Exception as e:
    print(f"An error occurred: {e}")
    print("Please double-check the base model name and your Vertex AI setup.")

In [21]:
print(f"Tuning job completed with Resource name: {sft_tuning_job.resource_name}")

Tuning job completed with Resource name: projects/677155171887/locations/us-central1/tuningJobs/5437787329584431104


In [ ]:
from vertexai.preview.tuning import sft
import vertexai
import time
import os
import json
from google.colab import auth
from google.cloud import aiplatform
from google.api_core import exceptions
from google.cloud.aiplatform_v1.types import JobState
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

## Evaluation

In [ ]:
import google.generativeai as genai
from rouge_score import rouge_scorer
import json
import os
from google.colab import userdata
import vertexai
from vertexai.preview.language_models import TextGenerationModel
from vertexai.preview import tuning
from vertexai.preview.tuning import TuningJob




# Retrieve API key from userdata
GOOGLE_API_KEY = userdata.get('GEMINI')

# Check if API key is retrieved successfully
if GOOGLE_API_KEY is None:
    raise ValueError("API key not found in userdata['GEMINI'].")

# Configure the client with the API key
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize Vertex AI (if needed)
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
REGION = os.environ.get("GOOGLE_CLOUD_REGION")  # Replace with your region if needed
vertexai.init(project=PROJECT_ID, location=REGION)

# Path to the evaluation dataset (JSONL format)
EVAL_DATASET_URI = f"gs://{BUCKET_NAME}/cmapss_FD004_test_text.jsonl"  # Replace with your actual dataset path


# Replace with the resource name you got after fine-tuning
tuned_model_resource_name = 'projects/677155171887/locations/us-central1/tuningJobs/5437787329584431104'

# Use TuningJob.get() to retrieve the tuning job
tuning_job = TuningJob.get(tuned_model_resource_name)  # Access the tuning job


# Get the best trained model from the tuning job
tuned_model_name = tuning_job.best_trial.trained_model

# Load the tuned model directly using TextGenerationModel.from_pretrained()
tuned_model = TextGenerationModel.from_pretrained(tuned_model_name)




# Load the tuned model using sft.TuningJob.get()
# Use sft.TuningJob to get the tuning job details
tuning_job = sft.TuningJob.get(tuned_model_resource_name)

# Load the tuned model directly using TextGenerationModel.from_pretrained()
tuned_model = TextGenerationModel.from_pretrained(tuned_model_resource_name)

# Function to calculate ROUGE scores
def calculate_rouge(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

# Evaluate the model
rouge_scores = []
with open(EVAL_DATASET_URI, 'r') as f:
    for line in f:
        try:
            data = json.loads(line)
            prompt = data["prompt"]
            reference = data["completion"]  # Expected completion

            # Generate text using the fine-tuned model
            response = tuned_model.predict(prompt=prompt)
            generated_text = response.text

            # Calculate ROUGE scores
            scores = calculate_rouge(reference, generated_text)
            rouge_scores.append(scores)

        except Exception as e:
            print(f"Error processing line: {line.strip()}, Error: {e}")

# Calculate average ROUGE scores
avg_rouge1 = sum([scores['rouge1'].fmeasure for scores in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([scores['rouge2'].fmeasure for scores in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([scores['rougeL'].fmeasure for scores in rouge_scores]) / len(rouge_scores)

# Print results
print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")